In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import Machine, generate_objects, BunchTracker, track, compare_traces, BeamTracker, plot3Dtraces
from test_tools import compare_beam_projections, plot_debug_data, plot_FIR_coefficients

from PyHEADTAIL_feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL_feedback.processors.multiplication import ChargeWeighter
from PyHEADTAIL_feedback.processors.linear_transform import Averager
from PyHEADTAIL_feedback.processors.misc import Bypass
from PyHEADTAIL_feedback.processors.register import Register, SerialRegister, TurnDelay, TurnFIRFilter
from PyHEADTAIL_feedback.processors.convolution import Lowpass, Gaussian, FIRFilter
from PyHEADTAIL_feedback.processors.resampling import DAC, HarmonicADC, BackToOriginalBins, Upsampler
from PyHEADTAIL_feedback.processors.addition import NoiseGenerator
np.random.seed(0)

# 010 Serial multibunch tracking

This tests injection error damping by using a model for the original LHC damper system (ADT). The model still in progress and might not be correct!

## Basic parameters and elements for the simulations

In [ ]:
%%capture

n_macroparticles = 1000
n_slices = 5
n_segments = 5
n_sigma_z = 6
n_sigma_z = 6

n_turns = 3
# n_turns = 12

machine = Machine(n_segments= n_segments)

# harmonic frequency of the bunches (f_RF/every_n_bucket_filled)
f_RF = 1./(machine.circumference/c/(float(machine.h_RF)))
f_harmonic = f_RF/10.


first_index = 10 #[buckets]
batch_spacing = 8  #[buckets]
# batch_spacing = 10  #[buckets]
n_batches = 3
# n_batches = 3
n_bunches_per_batch = 72
n_bunches_per_batch = 6
bunch_spacing = 1 #[buckets]
LHC_gap = 38
LHC_gap = 3

batch_separation = batch_spacing+n_bunches_per_batch* bunch_spacing

filling_scheme = []

for i in xrange(n_bunches_per_batch):
    filling_scheme.append(first_index + i * bunch_spacing)

first_index = np.max(filling_scheme) + LHC_gap
for j in xrange(n_batches):
    for i in xrange(n_bunches_per_batch):
        filling_scheme.append(first_index + i * bunch_spacing + j*batch_separation)


print filling_scheme

beam_ref, slicer_ref,trans_map, long_map = generate_objects(machine, n_macroparticles, n_slices,n_sigma_z,
                                                             filling_scheme=filling_scheme, matched=False)

In [ ]:
print 'f_RF: ' + str(f_RF)
print 'f_harmonic: ' + str(f_harmonic)
print('Number of bunches: ' + str(len(beam_ref.split())))

## Initial bunch kick
The bunhes are initial kicks are estimated from numerical MKI kicker waveform data. Data from Gerd Kotzian's archive.

In [ ]:
bunch_list = beam_ref.split()

n_bunches = len(bunch_list)

amplitude = 1e-3

for i in xrange(n_bunches):
        bunch_list[i].x = bunch_list[i].x + amplitude * (i + 1)
        bunch_list[i].y = bunch_list[i].y + amplitude * (i + 1)

beam_parallel = sum(bunch_list)

beam_serial = copy.deepcopy(bunch_list)

Determines holding scheme for the SignalHolder

## Feedback settings

In [ ]:
feedback_gain = 0.1
# feedback_gain = (0.1,0.4)

# delay (a number of turns) before the pickup signal is used to the correction kick calculations.
delay = 1

# a number of values used to calculate the correct signal
n_values = 2

RMS_noise_level = 1e-6


# feedback settings
fc=1e6 # The cut off frequency of the power amplifier
ADC_f = 40e9 # multiplier of the sampling rate from the harmonic frequency
ADC_n_samples = 50
ADC_bits = 16
ADC_range = (-3e-3,3e-3)

DAC_bits = 14
DAC_range = (-3e-3,3e-3)

RMS_noise_level = 1e-6

## Paralellel multi bunch tracking

In [ ]:
beam_ref_data = copy.deepcopy(beam_ref)
tracker_ref_data = BeamTracker(beam_ref_data)
slicer_ref_data = copy.deepcopy(slicer_ref)

parallel_register = Register(3,machine.Q_x,1)

processors_parallel_x = [
        Bypass(debug=False),
        parallel_register,
#         ChargeWeighter(normalization='segment_average', debug=True),
#         NoiseGenerator(RMS_noise_level, debug=True),
#         Gaussian(fc, normalization=('bunch_by_bunch', f_harmonic), debug=True),
#         TurnDelay(delay, machine.Q_x, n_values,debug=True)
]

processors_parallel_y = [
        Bypass(),
#         ChargeWeighter(normalization='segment_average'),
#         NoiseGenerator(RMS_noise_level),
#         Gaussian(fc, normalization=('bunch_by_bunch', f_harmonic)),
#         TurnDelay(delay, machine.Q_y, n_values)
]

feedback_map = OneboxFeedback(feedback_gain,slicer_ref_data,processors_parallel_x,processors_parallel_y, mpi=True)
one_turn_map = [i for i in trans_map] + [feedback_map] #  + [long_map]

for i in range(n_turns):
#        bunch_dump.dump()
    print beam_parallel.mean_x()

    for m_ in one_turn_map:
        m_.track(beam_parallel)


## Serial multi bunch tracking

In [ ]:
beam_ref_data = copy.deepcopy(beam_ref)
tracker_ref_data = BeamTracker(beam_ref_data)
slicer_ref_data = copy.deepcopy(slicer_ref)

serial_register = SerialRegister(3,machine.Q_x,1,n_bunches=len(filling_scheme))

processors_serial_x = [
        Bypass(debug=True),
        serial_register,
#         ChargeWeighter(normalization='segment_average', debug=True),
#         NoiseGenerator(RMS_noise_level, debug=True),
#         Gaussian(fc, normalization=('bunch_by_bunch', f_harmonic), debug=True),
#         TurnDelay(delay, machine.Q_x, n_values,debug=True)
]

processors_serial_y = [
        Bypass(),
#         ChargeWeighter(normalization='segment_average'),
#         NoiseGenerator(RMS_noise_level),
#         Gaussian(fc, normalization=('bunch_by_bunch', f_harmonic)),
#         TurnDelay(delay, machine.Q_y, n_values)
]

feedback_map = OneboxFeedback(feedback_gain,slicer_ref_data,processors_serial_x,processors_serial_y, mpi='serial')
one_turn_map = [i for i in trans_map] + [feedback_map] #  + [long_map]

for i in range(n_turns):
#        bunch_dump.dump()
    beam_temp = sum(beam_serial)
    print beam_temp.mean_x()

    for m_ in one_turn_map:
        for b in beam_serial:
            m_.track(b)

In [ ]:
fig, ax = plt.subplots(1)
print len(parallel_register)
for i, (param, signal, delay) in enumerate(parallel_register):
    if i==0:
        print param
    ax.plot(signal,'k-')

print len(serial_register)
for i, (param, signal, delay) in enumerate(serial_register):
    if i==0:
        print param
    ax.plot(signal,'r--')

### Turn-by-turn FIR filter for the betatron phase correction
The betatron phase correction is implemented manually by using a turn-by-turn FIR filter. 


Read more about the coefficients from Ref. http://accelconf.web.cern.ch/AccelConf/IPAC2011/papers/mopo013.pdf.

In [ ]:
# the total (group) delay to the middle coefficient is four turns, i.e.

turn_notch_filter = [-1.,1.]

phase_shift_x = -5. * machine.Q_x * 2.* pi
turn_phase_filter_x = [-2. * np.sin(phase_shift_x)/(pi * 3.),
                   0,
                   -2. * np.sin(phase_shift_x)/(pi * 1.),
                   np.cos(phase_shift_x),
                   2. * np.sin(phase_shift_x)/(pi * 1.),
                   0,
                   2. * np.sin(phase_shift_x)/(pi * 3.)
                   ]

phase_shift_y = -5. * machine.Q_y * 2.* pi
turn_phase_filter_y = [-2. * np.sin(phase_shift_y)/(pi * 3.),
                   0,
                   -2. * np.sin(phase_shift_y)/(pi * 1.),
                   np.cos(phase_shift_y),
                   2. * np.sin(phase_shift_y)/(pi * 1.),
                   0,
                   2. * np.sin(phase_shift_y)/(pi * 3.)
                   ]


turn_FIR_filter_x = np.convolve(turn_notch_filter, turn_phase_filter_x)
turn_FIR_filter_y = np.convolve(turn_notch_filter, turn_phase_filter_y)

# turn_FIR_filter_x = turn_phase_filter_x
# turn_FIR_filter_y = turn_phase_filter_y

### The model
This is the actual damper model to be used in the simulations, which includes elements for digital signal processing (ADC, FIR filters and DAC) and power amplifier/kicker bandwidth limitation. It is assumed that the signal transmission from beam to ADC is perfect. A detailed model for a pickup signal processing could be build, but it is not done here in order to clarify the example.

In [ ]:
beam_detailed = copy.deepcopy(beam_ref)
tracker_detailed = BeamTracker(beam_detailed)
slicer_detailed = copy.deepcopy(slicer_ref)

processors_detailed_x = [
        Bypass(debug=False),
        ChargeWeighter(normalization = 'segment_average', debug=False),
#         NoiseGenerator(RMS_noise_level, debug=False),
        HarmonicADC(1*f_RF/10., ADC_bits, ADC_range,
                    n_extras=extra_adc_bins, debug=False),
        TurnFIRFilter(turn_FIR_filter_x, machine.Q_x, delay=1, debug=False),
        FIRFilter(FIR_phase_filter, zero_tap = 40, debug=True),
#         SignalHolder(holding_scheme, debug=True),
        Upsampler(3, [0,3,0], debug=False),
        FIRFilter(FIR_gain_filter, zero_tap = 32, debug=True),
        DAC(ADC_bits, ADC_range, debug=False),
        Lowpass(fc, f_cutoff_2nd=10*fc, debug=True),
        BackToOriginalBins(debug=False),
]

processors_detailed_y = [
        Bypass(debug=True),
        ChargeWeighter(normalization = 'segment_average', debug=False),
#         NoiseGenerator(RMS_noise_level, debug=False),
        HarmonicADC(1*f_RF/10., ADC_bits, ADC_range,
                    n_extras=extra_adc_bins, debug=False),
        TurnFIRFilter(turn_FIR_filter_y, machine.Q_y, delay = 1, debug=True),
        FIRFilter(FIR_phase_filter, zero_tap = 41, debug=True),
#         SignalHolder(holding_scheme, debug=True),
        Upsampler(3, [1,1,1], debug=False),
        FIRFilter(FIR_gain_filter, zero_tap = 32, debug=True),
        DAC(ADC_bits, ADC_range, debug=False),
        Lowpass(fc, f_cutoff_2nd=10*fc, debug=False),
        BackToOriginalBins(debug=False),
]


feedback_map = OneboxFeedback(feedback_gain,slicer_detailed,
                              processors_detailed_x,processors_detailed_y, mpi=True)
one_turn_map = [feedback_map] + [i for i in trans_map] # + [long_map]

track(n_turns, beam_detailed, one_turn_map, tracker_detailed)
plot_debug_data(processors_detailed_x, source = 'output')

In [ ]:
# compare_traces([tracker_ref_data,tracker_detailed],
#                ['Ideal bunch-by-slice\nfeedback', 'Damper model'], bunch_idx=-10)
# compare_beam_projections([ beam_ref_data, beam_detailed], 
#                ['Ideal bunch-by-slice\nfeedback', 'Damper model'])

Jani Komppula, CERN, 2017

In [ ]:


plot3Dtraces(tracker_ref_data,
               ['Ideal bunch-by-slice\nfeedback'], bunches = 130,first_turn=0)

plot3Dtraces(tracker_detailed,
               ['Minimal model'], bunches = [100,120,203])